In [1]:
import json

import nltk
import pandas as pd

nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.tokenize import sent_tokenize

In [3]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
from typing import Any, Dict

In [4]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
tokenizer.pad_token = tokenizer.eos_token

/usr/lib/python3/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
muserc_train = pd.read_json("MuSeRC/train.jsonl", lines=True).to_dict(orient="records")
muserc_val = pd.read_json("MuSeRC/val.jsonl", lines=True).to_dict(orient="records")

In [6]:
def to_dg_format(dataset: list[dict[str, Any]]) -> list[dict[str, Any]]:
    dataset_processed = []
    item_id = 0

    for iidx, item in enumerate(dataset):
        for question in item["passage"]["questions"]:
            new_item = {
                "item_id": item_id,
                "passage_id": item["idx"],
                "passage": item["passage"]["text"],
                "question": question["question"],
                "distractors": [f'"{answer["text"]}"' for answer in question["answers"] if answer["label"] == 0],
                "right_answer": [answer["text"] for answer in question["answers"] if answer["label"] == 1][0]
            }
            dataset_processed.append(new_item)
            item_id += 1

    return dataset_processed

In [7]:
muserc_train_dg = to_dg_format(muserc_train)

In [8]:
muserc_train_dg[0]

{'item_id': 0,
 'passage_id': 0,
 'passage': '(1) Но люди не могут существовать без природы, поэтому в парке стояли железобетонные скамейки — деревянные моментально ломали. (2) В парке бегали ребятишки, водилась шпана, которая развлекалась игрой в карты, пьянкой, драками, «иногда насмерть». (3) «Имали они тут и девок...» (4) Верховодил шпаной Артемка-мыло, с вспененной белой головой. (5) Людочка сколько ни пыталась усмирить лохмотья на буйной голове Артемки, ничего у неё не получалось. (6) Его «кудри, издали напоминавшие мыльную пену, изблизя оказались что липкие рожки из вокзальной столовой — сварили их, бросили комком в пустую тарелку, так они, слипшиеся, неподъёмно и лежали. (7) Да и не ради причёски приходил парень к Людочке. (8) Как только её руки становились занятыми ножницами и расчёской, Артемка начинал хватать её за разные места. (9) Людочка сначала увёртывалась от хватких рук Артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на

In [9]:
muserc_val_dg = to_dg_format(muserc_val)

In [10]:
muserc_val_dg[0]

{'item_id': 0,
 'passage_id': 0,
 'passage': '(1) Самый первый «остров» Архипелага возник в 1923 году на месте Соловецкого монастыря. (2) Затем появились ТОНы — тюрьмы особого назначения и этапы. (3) Люди попадали на Архипелаг разными способами: в вагон-заках, на баржах, пароходах и пешими этапами. (4) В тюрьмы арестованных доставляли в «воронках» — фургончиках чёрного цвета. (5) Роль портов Архипелага играли пересылки, временные лагеря, состоящие из палаток, землянок, бараков или участков земли под открытым небом. (6) На всех пересылках держать «политических» в узде помогали специально отобранные урки, или «социально близкие». (7) Солженицын побывал на пересылке Красная Пресня в 1945 году. (8) Эмигранты, крестьяне и «малые народы» перевозили красными эшелонами. (9) Чаще всего такие эшелоны останав\xadливались на пустом месте, посреди степи или тайги, и осуждённые сами строили лагерь. (10) Особо важные заключённые, в основном учёные, перевозились спецконвоем. (11) Так перевозили и Солж

In [11]:
LB = "\n  "

def to_dg_format_final(dataset: list[dict[str, Any]]) -> list[dict[str, Any]]:
    new_dataset = []

    for item in dataset:
        new_item = {
            "item_id": item["item_id"],
            "passage_id": item["passage_id"],
            "inp": f'{item["passage"]} ВОПРОС: {item["question"]} ПРАВИЛЬНЫЙ ОТВЕТ: {item["right_answer"]} НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:{LB + LB.join(item["distractors"])}'
        }
        new_dataset.append(new_item)

    return new_dataset

In [12]:
muserc_train_dg = to_dg_format_final(muserc_train_dg)

In [13]:
muserc_val_dg = to_dg_format_final(muserc_val_dg)

In [14]:
muserc_train_dg[0]

{'item_id': 0,
 'passage_id': 0,
 'inp': '(1) Но люди не могут существовать без природы, поэтому в парке стояли железобетонные скамейки — деревянные моментально ломали. (2) В парке бегали ребятишки, водилась шпана, которая развлекалась игрой в карты, пьянкой, драками, «иногда насмерть». (3) «Имали они тут и девок...» (4) Верховодил шпаной Артемка-мыло, с вспененной белой головой. (5) Людочка сколько ни пыталась усмирить лохмотья на буйной голове Артемки, ничего у неё не получалось. (6) Его «кудри, издали напоминавшие мыльную пену, изблизя оказались что липкие рожки из вокзальной столовой — сварили их, бросили комком в пустую тарелку, так они, слипшиеся, неподъёмно и лежали. (7) Да и не ради причёски приходил парень к Людочке. (8) Как только её руки становились занятыми ножницами и расчёской, Артемка начинал хватать её за разные места. (9) Людочка сначала увёртывалась от хватких рук Артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на гол

In [15]:
muserc_val_dg[0]

{'item_id': 0,
 'passage_id': 0,
 'inp': '(1) Самый первый «остров» Архипелага возник в 1923 году на месте Соловецкого монастыря. (2) Затем появились ТОНы — тюрьмы особого назначения и этапы. (3) Люди попадали на Архипелаг разными способами: в вагон-заках, на баржах, пароходах и пешими этапами. (4) В тюрьмы арестованных доставляли в «воронках» — фургончиках чёрного цвета. (5) Роль портов Архипелага играли пересылки, временные лагеря, состоящие из палаток, землянок, бараков или участков земли под открытым небом. (6) На всех пересылках держать «политических» в узде помогали специально отобранные урки, или «социально близкие». (7) Солженицын побывал на пересылке Красная Пресня в 1945 году. (8) Эмигранты, крестьяне и «малые народы» перевозили красными эшелонами. (9) Чаще всего такие эшелоны останав\xadливались на пустом месте, посреди степи или тайги, и осуждённые сами строили лагерь. (10) Особо важные заключённые, в основном учёные, перевозились спецконвоем. (11) Так перевозили и Солжениц

In [16]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inp"]
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

In [17]:
muserc_train_dg = Dataset.from_list(muserc_train_dg)
muserc_val_dg = Dataset.from_list(muserc_val_dg)
len(muserc_train_dg), len(muserc_val_dg)

(2897, 529)

In [18]:
muserc_train_dg_tokenized = muserc_train_dg.map(preprocess_function, batched=True)
muserc_val_dg_tokenized = muserc_train_dg.map(preprocess_function, batched=True)

Map:   0%|          | 0/2897 [00:00<?, ? examples/s]

Map:   0%|          | 0/2897 [00:00<?, ? examples/s]

In [19]:
BATCH_SIZE  = 1
NUM_TRAIN_EPOCHS = 20
MODEL_NAME="RuGPT3-MuSeRC-DG"

args = TrainingArguments(
    output_dir=MODEL_NAME,
    evaluation_strategy="epoch", save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    prediction_loss_only=True,
    gradient_checkpointing=True
)

In [20]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [21]:
trainer = Trainer(
    model,
    args=args,
    train_dataset=muserc_train_dg_tokenized,
    eval_dataset=muserc_val_dg_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [22]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,1.150400,0.541531
2,0.484400,0.227905
3,0.279200,0.140868
4,0.206800,0.094576
5,0.151500,0.068186
6,0.106000,0.052249
7,0.087500,0.042251
8,0.072600,0.035998
9,0.060400,0.031186
10,0.052000,0.027906


TrainOutput(global_step=57940, training_loss=0.17629676591045576, metrics={'train_runtime': 7971.0343, 'train_samples_per_second': 7.269, 'train_steps_per_second': 7.269, 'total_flos': 1.225516787712e+16, 'train_loss': 0.17629676591045576, 'epoch': 20.0})